In [1]:
import torch
from dataset import CelebMBTIDataset

In [2]:
from utils import assign_splits, get_label_mapping

assign_splits(
    file_path="faces_yolo_metadata.csv",
    id_column="id",
    label_column="mbti",
)

mbti_to_idx = get_label_mapping(
    file_path="faces_yolo_metadata.csv",
    label_column="mbti",
)

for key in mbti_to_idx:
    if key.startswith("I"):
        mbti_to_idx[key] = 0
    else: 
        mbti_to_idx[key] = 1

print("MBTI to index mapping:", mbti_to_idx)

split
train    5609
test      715
val       694
Name: count, dtype: int64
MBTI to index mapping: {'ENFJ': 1, 'ENFP': 1, 'ENTJ': 1, 'ENTP': 1, 'ESFJ': 1, 'ESFP': 1, 'ESTJ': 1, 'ESTP': 1, 'INFJ': 0, 'INFP': 0, 'INTJ': 0, 'INTP': 0, 'ISFJ': 0, 'ISFP': 0, 'ISTJ': 0, 'ISTP': 0}


In [3]:
from torchvision import transforms

train_tf = transforms.Compose(
    [
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomApply([transforms.ColorJitter(0.25, 0.25, 0.25, 0.05)], p=0.8),
        transforms.RandomApply(
            [
                transforms.RandomAffine(
                    degrees=10,  # small rotation
                    translate=(0.05, 0.05),  # small shifts
                    scale=(0.9, 1.1),
                )
            ],
            p=0.5,
        ),
        transforms.RandomApply([transforms.GaussianBlur(kernel_size=3)], p=0.2),
        transforms.ToTensor(),
        transforms.RandomErasing(
            p=0.25, scale=(0.02, 0.1), ratio=(0.3, 3.3), value="random"
        ),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]
)

val_tf = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]
)

In [4]:
from torch.utils.data import DataLoader

train_ds = CelebMBTIDataset(
    root_dir="faces_yolo",
    metadata_csv="faces_yolo_metadata.csv",
    split="train",
    transform=train_tf,
    mbti_to_idx=mbti_to_idx,
)

val_ds = CelebMBTIDataset(
    root_dir="faces_yolo",
    metadata_csv="faces_yolo_metadata.csv",
    split="val",
    transform=val_tf,
    mbti_to_idx=mbti_to_idx,
)

test_ds = CelebMBTIDataset(
    root_dir="faces_yolo",
    metadata_csv="faces_yolo_metadata.csv",
    split="test",
    transform=val_tf,
    mbti_to_idx=mbti_to_idx,
)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_ds, batch_size=64, shuffle=False, num_workers=4)

for images, labels in train_loader:
    print(images.shape)  # [Batch, 3, 224, 224]
    print(labels.shape)  # [Batch, num_outputs]
    break

torch.Size([32, 3, 224, 224])
torch.Size([32])


In [ ]:
from torchvision.models import resnet18, ResNet18_Weights
import torch.nn as nn

num_classes = 2

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
in_features = model.fc.in_features

model.fc = nn.Sequential(
    nn.Dropout(p=0.5),
    nn.Linear(in_features, num_classes),
)

model.to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)

In [7]:
from tqdm import tqdm

for epoch in range(50):
    # training phase with tqdm
    pbar = tqdm(train_loader, desc=f"Epoch {epoch:02d} [train]", leave=False)
    model.train()
    total_loss, total_correct, total = 0.0, 0, 0
    for x, y in pbar:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        logits = model(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()

        preds = logits.argmax(1)
        total_correct += (preds == y).sum().item()
        total_loss += loss.item() * y.size(0)
        total += y.size(0)

        pbar.set_postfix(loss=loss.item())

    tr_loss = total_loss / total
    tr_acc = total_correct / total

    # validation phase with tqdm
    pbar = tqdm(val_loader, desc=f"Epoch {epoch:02d} [val]", leave=False)
    model.eval()
    total_loss, total_correct, total = 0.0, 0, 0
    with torch.no_grad():
        for x, y in pbar:
            x, y = x.to(device), y.to(device)
            logits = model(x)
            loss = criterion(logits, y)
            preds = logits.argmax(1)
            total_correct += (preds == y).sum().item()
            total_loss += loss.item() * y.size(0)
            total += y.size(0)

            pbar.set_postfix(loss=loss.item())

    va_loss = total_loss / total
    va_acc = total_correct / total

    print(
        f"Epoch {epoch:02d} | "
        f"train loss {tr_loss:.4f}/ train accuracy {tr_acc:.3f} | "
        f"val loss {va_loss:.4f}/ val accuracy {va_acc:.3f}"
    )

Epoch 00 | train loss 0.7688/ train accuracy 0.559 | val loss 0.6822/ val accuracy 0.588


Epoch 01 | train loss 0.7292/ train accuracy 0.579 | val loss 0.7218/ val accuracy 0.548


Epoch 02 | train loss 0.6986/ train accuracy 0.610 | val loss 0.6685/ val accuracy 0.617


Epoch 03 | train loss 0.6628/ train accuracy 0.629 | val loss 0.7096/ val accuracy 0.615


Epoch 04 | train loss 0.6334/ train accuracy 0.664 | val loss 0.7277/ val accuracy 0.578


...

Epoch 45 | train loss 0.2465/ train accuracy 0.984 | val loss 0.6316/ val accuracy 0.758


Epoch 46 | train loss 0.2460/ train accuracy 0.981 | val loss 0.6107/ val accuracy 0.755


Epoch 47 | train loss 0.2487/ train accuracy 0.981 | val loss 0.5946/ val accuracy 0.762


Epoch 48 | train loss 0.2454/ train accuracy 0.985 | val loss 0.5974/ val accuracy 0.768


Epoch 49 | train loss 0.2390/ train accuracy 0.988 | val loss 0.5790/ val accuracy 0.767
